In [ ]:
# Store Sales Time Series Forecasting - Data Exploration

This notebook explores the Favorita store sales dataset to understand:
1. Data structure and quality
2. Temporal patterns
3. Store and product relationships
4. External factors (oil prices, holidays)
5. Feature relationships and correlations


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set plotting style
plt.style.use('seaborn')
sns.set_palette('husl')


In [ ]:
def analyze_dataset(df, name):
    """
    Comprehensive analysis of a dataset including:
    - Basic info (size, memory usage)
    - Column details (types, unique values, missing values)
    - Descriptive statistics
    """
    print(f"\n{'='*50}")
    print(f"Analysis for: {name}")
    print(f"{'='*50}")
    
    # Basic Information
    print("\n1. Basic Information:")
    print(f"Number of Records: {len(df):,}")
    print(f"Number of Columns: {len(df.columns):,}")
    print(f"Memory Usage: {df.memory_usage(deep=True).sum() / (1024*1024):.2f} MB")
    
    # Column Analysis
    print("\n2. Column Details:")
    for col in df.columns:
        print(f"\nColumn: {col}")
        print(f"Type: {df[col].dtype}")
        print(f"Unique Values: {df[col].nunique():,}")
        print(f"Missing Values: {df[col].isnull().sum():,} ({(df[col].isnull().sum()/len(df))*100:.2f}%)")
        
        # For numeric columns
        if df[col].dtype in ['int64', 'float64']:
            print(f"Min: {df[col].min():,}")
            print(f"Max: {df[col].max():,}")
            print(f"Mean: {df[col].mean():.2f}")
        
        # For categorical/object columns
        elif df[col].dtype == 'object':
            if df[col].nunique() < 10:  # Only show if few unique values
                print("Value Counts:")
                print(df[col].value_counts().head())
    
    # Memory Optimization Suggestions
    print("\n3. Memory Optimization Suggestions:")
    for col in df.columns:
        if df[col].dtype == 'float64' and df[col].notnull().all():
            if df[col].round(0).equals(df[col]):
                print(f"- Column '{col}' could be converted to integer type")
        elif df[col].dtype == 'object':
            if df[col].nunique() / len(df) < 0.5:  # Less than 50% unique values
                print(f"- Column '{col}' could be converted to categorical type")

# Analyze each dataset
analyze_dataset(train_df, "Training Data")
analyze_dataset(stores_df, "Stores Data")
analyze_dataset(oil_df, "Oil Prices Data")
analyze_dataset(holidays_df, "Holidays Data")
analyze_dataset(transactions_df, "Transactions Data")


In [ ]:
### Data Dictionary

1. **Training Data (`train_df`)**:
   - `date`: Date of the record
   - `store_nbr`: Store identifier
   - `family`: Product family (category)
   - `sales`: Total sales for the product family at the store
   - `onpromotion`: Number of items in the product family on promotion

2. **Stores Data (`stores_df`)**:
   - `store_nbr`: Store identifier
   - `city`: City where the store is located
   - `state`: State where the store is located
   - `type`: Type of store
   - `cluster`: Store cluster based on similarity

3. **Oil Prices Data (`oil_df`)**:
   - `date`: Date of the oil price record
   - `dcoilwtico`: Daily price of oil in USD

4. **Holidays Data (`holidays_df`)**:
   - `date`: Date of the holiday/event
   - `type`: Type of holiday (Holiday, Transfer, Additional, Bridge, Work Day)
   - `locale`: National, Regional, or Local
   - `locale_name`: Specific region if Regional/Local
   - `description`: Description of the holiday
   - `transferred`: Boolean indicating if the holiday was transferred

5. **Transactions Data (`transactions_df`)**:
   - `date`: Date of transactions
   - `store_nbr`: Store identifier
   - `transactions`: Number of transactions on the given date


In [ ]:
# Time Range Analysis
print("Dataset Time Ranges:")
print("\nTraining Data:")
print(f"Start Date: {train_df['date'].min()}")
print(f"End Date: {train_df['date'].max()}")
print(f"Total Days: {(train_df['date'].max() - train_df['date'].min()).days}")

print("\nOil Prices Data:")
print(f"Start Date: {oil_df['date'].min()}")
print(f"End Date: {oil_df['date'].max()}")
print(f"Total Days: {(oil_df['date'].max() - oil_df['date'].min()).days}")

print("\nTransactions Data:")
print(f"Start Date: {transactions_df['date'].min()}")
print(f"End Date: {transactions_df['date'].max()}")
print(f"Total Days: {(transactions_df['date'].max() - transactions_df['date'].min()).days}")

# Check for data completeness
print("\nData Completeness Check:")
total_days = (train_df['date'].max() - train_df['date'].min()).days + 1
expected_records = total_days * len(train_df['store_nbr'].unique()) * len(train_df['family'].unique())
actual_records = len(train_df)
print(f"\nTraining Data:")
print(f"Expected Records: {expected_records:,}")
print(f"Actual Records: {actual_records:,}")
print(f"Completeness: {(actual_records/expected_records)*100:.2f}%")


In [ ]:
# Data Quality Checks

def check_data_quality(df, date_col='date'):
    """
    Perform various data quality checks on the dataset
    """
    print(f"\nData Quality Checks:")
    
    # 1. Check for duplicates
    duplicates = df.duplicated().sum()
    print(f"\n1. Duplicate Records: {duplicates:,} ({(duplicates/len(df))*100:.2f}%)")
    
    # 2. Check for negative values in numeric columns
    print("\n2. Negative Values Check:")
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    for col in numeric_cols:
        neg_count = (df[col] < 0).sum()
        if neg_count > 0:
            print(f"   - {col}: {neg_count:,} negative values")
    
    # 3. Check for date continuity if date column exists
    if date_col in df.columns:
        print("\n3. Date Continuity Check:")
        dates = pd.date_range(start=df[date_col].min(), end=df[date_col].max())
        missing_dates = set(dates) - set(df[date_col].unique())
        if missing_dates:
            print(f"   - Missing dates: {len(missing_dates)}")
            print(f"   - First few missing dates: {sorted(list(missing_dates))[:5]}")
        else:
            print("   - No missing dates found")
    
    # 4. Check for outliers in numeric columns using IQR method
    print("\n4. Outlier Detection (IQR method):")
    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = ((df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))).sum()
        if outliers > 0:
            print(f"   - {col}: {outliers:,} outliers ({(outliers/len(df))*100:.2f}%)")

# Run quality checks on each dataset
print("Quality Check - Training Data")
check_data_quality(train_df)

print("\nQuality Check - Oil Prices Data")
check_data_quality(oil_df)

print("\nQuality Check - Transactions Data")
check_data_quality(transactions_df)


In [ ]:
### Summary of Initial Data Analysis

1. **Dataset Sizes and Structure**
   - Training data spans multiple years with daily records
   - Each record represents sales for a specific product family at a specific store
   - Multiple supplementary datasets provide context (stores, oil prices, holidays)

2. **Key Metrics**
   - Number of stores
   - Number of product families
   - Date range coverage
   - Transaction volumes
   - Sales distributions

3. **Data Quality Findings**
   - Completeness of records
   - Presence of outliers
   - Missing value patterns
   - Date continuity
   - Data type consistency

4. **Potential Challenges**
   - Handling missing values
   - Dealing with outliers
   - Date alignment across datasets
   - Memory optimization opportunities

5. **Next Steps**
   - Detailed temporal analysis
   - Store-level analysis
   - Product family analysis
   - External factors correlation study


In [ ]:
# Load all datasets
train_df = pd.read_csv('../dataset/store-sales-time-series-forecasting/train.csv')
stores_df = pd.read_csv('../dataset/store-sales-time-series-forecasting/stores.csv')
oil_df = pd.read_csv('../dataset/store-sales-time-series-forecasting/oil.csv')
holidays_df = pd.read_csv('../dataset/store-sales-time-series-forecasting/holidays_events.csv')
transactions_df = pd.read_csv('../dataset/store-sales-time-series-forecasting/transactions.csv')

# Convert date columns
train_df['date'] = pd.to_datetime(train_df['date'])
oil_df['date'] = pd.to_datetime(oil_df['date'])
holidays_df['date'] = pd.to_datetime(holidays_df['date'])
transactions_df['date'] = pd.to_datetime(transactions_df['date'])
